In [33]:
import requests
import threading
from http.server import BaseHTTPRequestHandler, HTTPServer
import webbrowser

authorization_code = None

class AuthorizationHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        global authorization_code
        if self.path.startswith("/?code"):
            authorization_code = self.path.split("/?code=")[1]

            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()
            self.wfile.write(b"<h2>Authorization Code Received, Close this tab and go back to the session</h2>")

            raise SystemExit

In [34]:
from requests_oauthlib import OAuth2Session

client_id = "e3f5090e15587e7ec529"
client_secret = "1e7409f6b2140a5a3cec1f07d472bc515d5bce28"
authorization_base_url = 'https://github.com/login/oauth/authorize'
token_url = 'https://github.com/login/oauth/access_token'
redirect_uri = "http://localhost:1410"
callback_uri = redirect_uri

In [35]:
def start_temp_server():
    server = HTTPServer(("localhost", 1410), AuthorizationHandler)
    server.serve_forever()

github = OAuth2Session(client_id)
authorization_url, state = github.authorization_url(authorization_base_url)

In [36]:
authorization_url, state

('https://github.com/login/oauth/authorize?response_type=code&client_id=e3f5090e15587e7ec529&state=vUzY3oN8E7WVRn8fy5crN5PxEe7jC5',
 'vUzY3oN8E7WVRn8fy5crN5PxEe7jC5')

In [37]:
server_thread = threading.Thread(target=start_temp_server)
server_thread.start()
webbrowser.open(authorization_url)
server_thread.join()

127.0.0.1 - - [05/May/2024 08:33:03] "GET /?code=55d254720451071e58f0&state=vUzY3oN8E7WVRn8fy5crN5PxEe7jC5 HTTP/1.1" 200 -


In [38]:
authorization_code

'55d254720451071e58f0&state=vUzY3oN8E7WVRn8fy5crN5PxEe7jC5'

In [39]:
headers = {'accept':'*/*'}
parameters = {'client_id':client_id,'client_secret':client_secret,'access_code':authorization_code}
response = requests.get(token_url,headers=headers,params=parameters)
print(response.text)
token = response.text

error=bad_verification_code&error_description=The+code+passed+is+incorrect+or+expired.&error_uri=https%3A%2F%2Fdocs.github.com%2Fapps%2Fmanaging-oauth-apps%2Ftroubleshooting-oauth-app-access-token-request-errors%2F%23bad-verification-code


In [40]:
import github as gh

g = gh.Github()
app = g.get_oauth_application(client_id, client_secret)
token = app.get_access_token(authorization_code)


# token = github.fetch_token(
#     token_url,
#     code=authorization_code,
#     client_secret=client_secret)

BadCredentialsException: 200 {"error": "bad_verification_code", "error_description": "The code passed is incorrect or expired.", "error_uri": "https://docs.github.com/apps/managing-oauth-apps/troubleshooting-oauth-app-access-token-request-errors/#bad-verification-code"}